In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#The csv file that I got after trimming down to the birds I was interested is still 550 MB, but much more manageable than 31 GB
#This data is very clean, the main challenge was working with a file this large and keeping only the data I need


In [6]:
bird_types = ['Hawk', 'Eagle', 'Kestrel', 'kestrel', 'eagle', 'Falcon', 'falcon', 'hawk', 'Owl', 'owl sp.', 'Harrier', 'harrier', 'Kite', 'kite', 'Osprey', 'Merlin']
with open("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/ebd_US-CA_relDec-2021.txt") as file:
    df = pd.DataFrame()
    chunksize = 10 ** 4
    with pd.read_csv(file,  sep = '\t', header = 0, chunksize=chunksize) as reader:
        for chunk in reader:
            chunk.drop(chunk[~chunk['COMMON NAME'].str.contains('|'.join(bird_types))].index, inplace = True)
            relevant_chunk = (chunk[["LAST EDITED DATE", "COMMON NAME", "SCIENTIFIC NAME", "COUNTY", "COUNTY CODE", "LATITUDE", "LONGITUDE", "OBSERVATION DATE"]])
            df = df.append(relevant_chunk)

KeyboardInterrupt: 

In [ ]:
print(df)

In [ ]:
save_file = "/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey.csv"
df.to_csv(save_file)

In [3]:
with open("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey.csv") as csv_file:

    df = pd.read_csv(csv_file)
    non_raptors = ['Common Nighthawk', 'Lesser Nighthawk', 'nighthawk', 'waterfowl'] # Removing non-raptor species that remained in the dataset
    df.drop(df[df['COMMON NAME'].str.contains('|'.join(non_raptors))].index, inplace = True)
    # a few values are not specific species but 'falcon sp.' or similar.
    non_specific = ['hybrid', 'sp.', 'Eurasian Kestrel', '/']
    df.drop(df[df['COMMON NAME'].str.contains('|'.join(non_specific))].index, inplace = True)
    birdlist = df["COMMON NAME"].unique()
    print(len(birdlist))

36


In [6]:
#Data processing done in advance of ML part of the project. I am adding a species description dataset so that
#predictive analysis can be done later based on where a bird was seen and what color it was

descriptions_csv = "/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Bird_descriptions.csv"
df_desc = pd.read_csv(descriptions_csv)
df_desc = df_desc[['Bird', 'Size', 'Main Colors', 'Category']]
df_desc["Brown"] = np.where(df_desc["Main Colors"].str.contains("brown"), 1, 0)
df_desc["White"] = np.where(df_desc["Main Colors"].str.contains("white"), 1, 0)
df_desc["Gray"] = np.where(df_desc["Main Colors"].str.contains("gray"), 1, 0)
df_desc["Orange"] = np.where(df_desc["Main Colors"].str.contains("orange"), 1, 0)
df_desc["Black"] = np.where(df_desc["Main Colors"].str.contains("black"), 1, 0)

df.head()

,Unnamed: 0,LAST EDITED DATE,COMMON NAME,SCIENTIFIC NAME,COUNTY,COUNTY CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,Size_x,...,White_y,Gray_y,Orange_y,Black_y,Hawk,Falcon,Kite,Eagle,Owl,Kestrel
0,0,2021-08-05 02:45:57.629774,Bald Eagle,Haliaeetus leucocephalus,Del Norte,US-CA-015,41.754666,-124.202714,1999-01-18,goose-sized or larger,...,1,0,0,0,0,0,0,1,0,0
1,1,2021-04-01 02:35:26.368505,Cooper's Hawk,Accipiter cooperii,Riverside,US-CA-065,33.531253,-117.075205,1996-10-13,crow-sized,...,0,1,1,0,1,0,0,0,0,0
2,2,2021-04-01 02:35:26.368505,Red-tailed Hawk,Buteo jamaicensis,Riverside,US-CA-065,33.462797,-115.980082,1997-08-22,between crow and goose,...,1,0,1,0,1,0,0,0,0,0
3,3,2017-08-16 00:55:22,American Kestrel,Falco sparverius,Kern,US-CA-029,35.615712,-118.251686,1995-09-11,robin-sized,...,0,1,1,1,0,0,0,0,0,1
4,4,2014-10-19 02:09:50,White-tailed Kite,Elanus leucurus,Monterey,US-CA-053,36.826080,-121.632273,1994-10-15,between crow and goose,...,1,1,0,1,0,0,1,0,0,0


In [7]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Bird'], axis=1, inplace=True)
df.head()

KeyError: "['Unnamed: 0.1'] not found in axis"

In [7]:
descriptions_csv = "/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Bird_descriptions.csv"
df_categories = pd.read_csv(descriptions_csv)
df_categories["Hawk"] = np.where(df_categories["Category"].str.contains("Hawk"), 1, 0)
df_categories["Falcon"] = np.where(df_categories["Category"].str.contains("Falcon"), 1, 0)
df_categories["Kite"] = np.where(df_categories["Category"].str.contains("Kite"), 1, 0)
df_categories["Eagle"] = np.where(df_categories["Category"].str.contains("Eagle"), 1, 0)
df_categories["Owl"] = np.where(df_categories["Category"].str.contains("Owl"), 1, 0)
df_categories["Kestrel"] = np.where(df_categories["Category"].str.contains("Kestrel"), 1, 0)
df = df.merge(df_categories[['Bird', 'Category', 'Hawk', 'Falcon', 'Kite', 'Eagle', 'Owl', 'Kestrel']], 
              how = 'left', left_on = 'COMMON NAME', right_on = 'Bird')
df.head()

,Unnamed: 0,LAST EDITED DATE,COMMON NAME,SCIENTIFIC NAME,COUNTY,COUNTY CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,Size,...,Orange,Black,Bird,Category,Hawk,Falcon,Kite,Eagle,Owl,Kestrel
0,0,2021-08-05 02:45:57.629774,Bald Eagle,Haliaeetus leucocephalus,Del Norte,US-CA-015,41.754666,-124.202714,1999-01-18,goose-sized or larger,...,0,0,Bald Eagle,Eagle,0,0,0,1,0,0
1,1,2021-04-01 02:35:26.368505,Cooper's Hawk,Accipiter cooperii,Riverside,US-CA-065,33.531253,-117.075205,1996-10-13,crow-sized,...,1,0,Cooper's Hawk,Hawk,1,0,0,0,0,0
2,2,2021-04-01 02:35:26.368505,Red-tailed Hawk,Buteo jamaicensis,Riverside,US-CA-065,33.462797,-115.980082,1997-08-22,between crow and goose,...,1,0,Red-tailed Hawk,Hawk,1,0,0,0,0,0
3,3,2017-08-16 00:55:22,American Kestrel,Falco sparverius,Kern,US-CA-029,35.615712,-118.251686,1995-09-11,robin-sized,...,1,1,American Kestrel,Kestrel,0,0,0,0,0,1
4,4,2014-10-19 02:09:50,White-tailed Kite,Elanus leucurus,Monterey,US-CA-053,36.826080,-121.632273,1994-10-15,between crow and goose,...,0,1,White-tailed Kite,Kite,0,0,1,0,0,0


In [8]:
save_file = "/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey.csv"
df.to_csv(save_file)

In [9]:
df.head()

,Unnamed: 0,LAST EDITED DATE,COMMON NAME,SCIENTIFIC NAME,COUNTY,COUNTY CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,Size,...,Orange,Black,Bird,Category,Hawk,Falcon,Kite,Eagle,Owl,Kestrel
0,0,2021-08-05 02:45:57.629774,Bald Eagle,Haliaeetus leucocephalus,Del Norte,US-CA-015,41.754666,-124.202714,1999-01-18,goose-sized or larger,...,0,0,Bald Eagle,Eagle,0,0,0,1,0,0
1,1,2021-04-01 02:35:26.368505,Cooper's Hawk,Accipiter cooperii,Riverside,US-CA-065,33.531253,-117.075205,1996-10-13,crow-sized,...,1,0,Cooper's Hawk,Hawk,1,0,0,0,0,0
2,2,2021-04-01 02:35:26.368505,Red-tailed Hawk,Buteo jamaicensis,Riverside,US-CA-065,33.462797,-115.980082,1997-08-22,between crow and goose,...,1,0,Red-tailed Hawk,Hawk,1,0,0,0,0,0
3,3,2017-08-16 00:55:22,American Kestrel,Falco sparverius,Kern,US-CA-029,35.615712,-118.251686,1995-09-11,robin-sized,...,1,1,American Kestrel,Kestrel,0,0,0,0,0,1
4,4,2014-10-19 02:09:50,White-tailed Kite,Elanus leucurus,Monterey,US-CA-053,36.826080,-121.632273,1994-10-15,between crow and goose,...,0,1,White-tailed Kite,Kite,0,0,1,0,0,0


In [64]:
print(df[df["LATITUDE"].isnull() == True].shape) #no missing data
print(df[df["LONGITUDE"].isnull() == True].shape) #no missing data
print(df[df["COUNTY"].isnull() == True].shape)  #County is missing 3 values
print(df[df["OBSERVATION DATE"].isnull() == True].shape) #no missing data
print(df[df["COUNTY CODE"].isnull() == True].shape) #County Code is missing 3 values
print(df[df["LAST EDITED DATE"].isnull() == True].shape) #no missing data

#The three lines missing county data will be kept for Lat/Long analysis, but removed for county-level analysis

#ebird allows people to add historical data from before ebird existed, so data prior to 2002 is still valid. 
#During the EDA phase I will look into how far back we want to go, since data is sparse in the years prior to 1980

# print(df[pd.DatetimeIndex(df["OBSERVATION DATE"]).year <= 1980 ])
print(df["OBSERVATION DATE"].min())
print(df["OBSERVATION DATE"].max())

(0, 9)
(0, 9)
(3, 9)
(0, 9)
(3, 9)
(0, 9)
1860-08-23
2021-12-31


In [ ]:
#